In [3]:
from unsloth import FastLanguageModel
from transformers import pipeline
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/miniconda3/envs/grpo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-11 19:10:22 [__init__.py:239] Automatically detected platform cuda.


2025-05-11 19:10:23,333	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:

# Replace with your actual model directory
model_dir = "/root/project/GRPO-Ozaki/output/llmjp-grpo-trained/final_model"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_dir,
    max_seq_length=1024,
    load_in_4bit=False,
    fast_inference=False,  # 👈 disable fast inference
)

# Put model in eval mode
model.eval()

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.2.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 4. Max memory: 79.19 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 6/6 [00:04<00:00,  1.49it/s]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(99584, 5120, padding_idx=4)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
    

In [14]:
data_path = "/root/project/GRPO-Ozaki/data/results/answers_ENFP_nemotron_with_results.json"

with open(data_path, "r") as f:
    data = json.load(f)
data[0].keys()


dict_keys(['question_id', 'generated_question', 'answer', 'llm_as_judge_inference', 'llmjp-13b_instruct'])

In [21]:
import json
import re

data_path = "/root/project/GRPO-Ozaki/data/results/answers_ENFP_nemotron_with_results.json"

# JSONファイルを読み込む
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 最初の100件に対して "Assistant:" 以降を抽出
for i in data[:100]:
    full_output = i.get("llm_as_judge_inference", "")
    match = re.search(r"Assistant:\s*(.*)", full_output, re.DOTALL)
    if match:
        i["llm_as_judge_inference"] = match.group(1).strip()

# 元のファイルに上書き保存する
with open(data_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"更新内容を {data_path} に保存しました。")

更新内容を /root/project/GRPO-Ozaki/data/results/answers_ENFP_nemotron_with_results.json に保存しました。


In [ ]:


system_prompt = \
f"""あなたは質問に回答する優秀なアシスタントです.
話し方・価値観・思考パターンを反映した、自然かつ一貫性のある回答をして下さい。"""

def build_prompt(messages):
    return "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])

def infer(question: str):
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    input_text = build_prompt(prompt)

    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    output_ids = model.generate(
        **inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,  # just in case
    )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

# Example
prediction =  infer(data[0]["generated_question"])
print(prediction)


System: あなたは質問に回答する優秀なアシスタントです.
話し方・価値観・思考パターンを反映した、自然かつ一貫性のある回答をして下さい。
User: 持続可能な生活に関心がある場合、具体的な行動計画を立てることを重視しますか、それとも環境への影響について広範な認識を持つことを優先しますか？

Assistant: 私は持続可能な生活へのアプローチとして、両方の側面が重要だと考えています。まず、環境への影響について広範な認識を持つことは、持続可能な生活を実現するための基盤となります。これにより、日常の行動が地球環境にどのような影響を与えるのかを理解し、その影響を最小限に抑えるための具体的な行動を計画することが可能になります。例えば、エネルギーの使用効率を高めることや、リサイクルの習慣を身につけること、そして持続可能な製品を選ぶことなどが挙げられます。

ただし、広範な認識だけでは不十分で、具体的な行動計画がなければ持続可能な生活の実現は難しいでしょう。行動計画を立てる際には、短期的および長期的な目標を設定し、それを達成するためのステップを明確にすることが重要です。また、定期的に進捗を評価し、必要に応じて計画を修正することも求められます。

したがって、私は環境への影響についての広範な認識を持ちつつ、それに基づいた具体的な行動計画を立てることが最も効果的なアプローチだと考えています。このバランスが取れたアプローチにより、持続可能な生活をより効果的に実現できるでしょう。


In [19]:
data[0]["answer"]

'私が答える前に、自分の性格タイプを再確認してみましょう。\n\n*   自由奔放で社交的\n*   情熱的で創造的\n*   直感を重視し、人生の深い意味や目的を探求\n*   自立心が強く、興味を持ったことには夢中になる\n*   感受性が高く、他者の気持ちに共感\n\n持続可能な生活に関心がある場合、具体的な行動計画を立てることと環境への影響について広範な認識を持つことの両方は重要ですが、私の性格タイプに基づいて優先順位を付けるなら、**環境への影響について広範な認識を持つことを優先します**。その理由は以下の通りです。\n\n1.  **直感と創造性の活用**：広範な認識を優先することで、私の直感と創造性を活用して、持続可能な生活のより広い視点を把握できます。環境への影響を理解することで、革新的な解決策を見出すきっかけを得られます。\n2.  **深い意味や目的の探求**：環境への影響についての認識は、持続可能な生活の本質と人生におけるその意味をより深く理解するのに役立ちます。持続可能な生活が私自身や社会、地球に与える影響を考えることで、生活に深い目的意識を感じることができます。\n3.  **感受性と共感**：環境への影響を理解することで、自然や他の生き物とのつながりを深めることができます。これは、私の感受性と共感を高め、持続可能な生活への関心をさらに強めることにつながります。\n4.  **自立心と情熱**：広範な認識を得た後、自立心を活かし、自分の生活スタイルに合わせた具体的な行動計画を立てることができます。環境への影響を理解していることで、行動の重要性をより強く感じ、持続可能な生活への情熱が持続します。\n\n具体的な行動計画も重要ですが、私の性格タイプに合わせると、まずは環境への影響についての認識を深めることで、創造的にかつ情熱を持って持続可能な生活を送る基盤を作りたいと思います。そうすることで、自然な流れで自立心を活かし、具体的な行動計画を立てるようになるでしょう。'

In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("DeL-TaiseiOzaki/50_mbti_test")

/root/miniconda3/envs/grpo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 50/50 [00:00<00:00, 7679.34 examples/s]


In [13]:
ds["train"][0]

{'question_id': 1,
 'question_text': '社会的な集まりでは、大人数と交流するほうがエネルギーを得られますか、それとも1対1で話すほうがエネルギーを得られますか？',
 'option_a': {'text': '大人数と交流する', 'trait': 'E'},
 'option_b': {'text': '1対1で話す', 'trait': 'I'}}